In [1]:
import os
import numpy as np
import sys
print(sys.executable)
import tensorflow as tf
from tensorflow.keras import layers, models
import import_ipynb
import sys
sys.path.insert(1, '../src/')
from config import raw_data_path, univariate_data_path, processed_data_path


/bin/python3.11


2025-03-13 13:50:54.826238: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1741870254.903638 3634314 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1741870254.920513 3634314 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-13 13:50:55.457928: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
# print number of instances missing preterm label

# Load the preprocessed dataset
file_path = os.path.join(processed_data_path, 'target_preprocessed.npy')  # Update this with the actual path
target_data = np.load(file_path, allow_pickle=True)

# Count total instances
total_instances = len(target_data)

# Count instances where 'preterm' is None
none_preterm_count = sum(1 for entry in target_data if entry['preterm'] is None)

# Print results
print(f"Total instances: {total_instances}")
print(f"Instances with 'preterm' as None: {none_preterm_count}")


Total instances: 143
Instances with 'preterm' as None: 0


In [9]:

class Classifier:
    def __init__(self, encoder, input_shape, num_classes):
        self.encoder = encoder
        self.model = self._build_classifier(input_shape, num_classes)
    
    def _build_classifier(self, input_shape, num_classes):
        inputs = layers.Input(shape=input_shape)

        # Freeze encoder weights
        self.encoder.trainable = False

        x = self.encoder(inputs, training=False)  # Use frozen encoder
        x = layers.Dense(32, activation="relu")(x)
        x = layers.Dense(16, activation="relu")(x)
        outputs = layers.Dense(num_classes, activation="softmax")(x)

        model = models.Model(inputs, outputs)
        model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
                    loss="sparse_categorical_crossentropy",
                    metrics=["accuracy"])
        return model


    def train(self, x_train, y_train, batch_size=64, epochs=50, validation_split=0.1):
        history = self.model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=validation_split)
        return history
    
    def evaluate(self, x_test, y_test):
        return self.model.evaluate(x_test, y_test)
    
    def predict(self, x):
        return self.model.predict(x)

# # Load the trained encoder from MaskedAutoencoder
# autoencoder = MaskedAutoencoder(input_dim=500)  # Use the correct input dimension

# # Define an input layer matching the expected shape
# input_layer = layers.Input(shape=(500,))  # Make sure this matches your signal window size

# # Pass the input through the autoencoder's encoder
# encoded_output = autoencoder.encoder(input_layer)

# # Create a new model for the encoder
# encoder = models.Model(input_layer, encoded_output)

encoder_path = "../models/encoder_model.keras"

if os.path.exists(encoder_path):
    # Load saved encoder
    print("Loading saved encoder...")
    encoder = tf.keras.models.load_model(encoder_path)
else:
    # Train autoencoder and extract encoder
    autoencoder = MaskedAutoencoder(input_dim=500)
    input_layer = layers.Input(shape=(500,))
    encoded_output = autoencoder.encoder(input_layer)
    encoder = models.Model(input_layer, encoded_output)

    # Save encoder
    print("Saving trained encoder...")
    encoder.save(encoder_path)

# Load the target dataset
save_dir = os.path.join(univariate_data_path, "target_univariate.npy")
target_data = np.load(save_dir, allow_pickle=True)

def create_windows(data, window_size=500, step=250):
    all_windows, labels = [], []
    for sample in data:
        signal, label = sample["signal"], sample["preterm"]
        
        # Ensure the label is a valid integer
        if label is None:  # Skip instances with None labels (just in case)
            continue
        
        for start in range(0, len(signal) - window_size + 1, step):
            window = signal[start : start + window_size]
            all_windows.append(window)
            labels.append(label)

    return np.array(all_windows, dtype=np.float32), np.array(labels, dtype=np.int32)

x_train, y_train = create_windows(target_data)  # Make sure this function is called!


print(type(y_train[0]))
# Initialize and train the classifier
num_classes = len(set(y_train))  # Get number of unique classes
classifier = Classifier(encoder, input_shape=(500,), num_classes=num_classes)
print(f"x_train type: {type(x_train)}, dtype: {x_train.dtype}, shape: {x_train.shape}")
print(f"y_train type: {type(y_train)}, dtype: {y_train.dtype}, shape: {y_train.shape}")

classifier.train(x_train, y_train)


Loading saved encoder...
<class 'numpy.int32'>
x_train type: <class 'numpy.ndarray'>, dtype: float32, shape: (608832, 500)
y_train type: <class 'numpy.ndarray'>, dtype: int32, shape: (608832,)
Epoch 1/50
8562/8562 ━━━━━━━━━━━━━━━━━━━━ 11s 1ms/step - accuracy: 0.7261 - loss: 0.5957 - val_accuracy: 0.6906 - val_loss: 0.6570
Epoch 2/50
8562/8562 ━━━━━━━━━━━━━━━━━━━━ 10s 1ms/step - accuracy: 0.7354 - loss: 0.5397 - val_accuracy: 0.7141 - val_loss: 0.6294
Epoch 3/50
8562/8562 ━━━━━━━━━━━━━━━━━━━━ 10s 1ms/step - accuracy: 0.7369 - loss: 0.5344 - val_accuracy: 0.7037 - val_loss: 0.6526
Epoch 4/50
8562/8562 ━━━━━━━━━━━━━━━━━━━━ 10s 1ms/step - accuracy: 0.7371 - loss: 0.5318 - val_accuracy: 0.7105 - val_loss: 0.6408
Epoch 5/50
8562/8562 ━━━━━━━━━━━━━━━━━━━━ 10s 1ms/step - accuracy: 0.7386 - loss: 0.5288 - val_accuracy: 0.7125 - val_loss: 0.6399
Epoch 6/50
8562/8562 ━━━━━━━━━━━━━━━━━━━━ 10s 1ms/step - accuracy: 0.7396 - loss: 0.5270 - val_accuracy: 0.7107 - val_loss: 0.6306
Epoch 7/50
8562/8562 

In [11]:
# Define save path
classifier_save_path = "../models/classifier_model.keras"

# Save the trained classifier
classifier.model.save(classifier_save_path)

print(f"Classifier model saved at: {classifier_save_path}")

Classifier model saved at: ../models/classifier_model.keras
